# Tutorial 1: The Model
The `pycotools.model.Model` class is of central importance in pycotools. Copasi models are parsed from the copasi xml into custom python classes for storing model components such as metabolites, global quantities or reactions.  

### Imports

In [1]:
import site
site.addsitedir('//home/b3053674/Documents/pycotools')
from pycotools import *
from pycotools.Tests import test_models
import os
from lxml import etree

root:INFO:27:    Initializing pycotools
root:INFO:28:    Initializing logging System
root:INFO:29:    logging config file at: //home/b3053674/Documents/pycotools/pycotools/logging_config.conf


## Get Example Model
The Tests folder of the pycotools distribution is a mini python package containing a single importable file called test_models. The test_models.py module is simply a container for a collection of copasi models in xml format. These models are used throughout the tutorials. 

### Test models
-  Zi et al 2012
-  Kholodenko 2000

In [2]:
## get string model from test_models
zi_model_string = test_models.TestModels().zi_model()

## get a working directory. Change this to change this to wherever you like
directory = r'/home/b3053674/Documents/pycotools/ZiModel'

## choose path to zi model
zi_path = os.path.join(directory, 'zi2012.cps')

##write model to file
with open(zi_path, 'w') as f:
    f.write(zi_model_string)
    
## check file exists
if not os.path.isfile(zi_path):
    raise Exception



# Parse a Model into Pycotools
The model.Model class accepts a path to a existing copasi file

In [3]:
zi = model.Model(zi_path)
zi

Model(name=Zi2007_TGFbeta_signaling, time_unit=min, volume_unit=l, quantity_unit=nmol)

# Open and Save
It is sometimes desirable to `open` a copasi model that you're working with to manually perform sanity checks. The Model class has the open method for convenience: 

In [4]:
zi.open()

The `Model.open()` method implicitly saves to file before calling CopasiUI form the command line. This means that the environment variable CopasiUI must be set and pointing to the location of the CopasiUI.exe (this is usually done when copasi installs). By default, the model is overwritten but this behaviour can be modified by passing a new filename to the `copasi_file` argument:

In [5]:
## modify the zi path
new_path = zi_path[:-4]+'2.cps'

## saves model to new_path then opens
zi.open(copasi_file=new_path)

The `Model.save()` behaves similarly to `Model.open`. By default it overwrites but takes an optional `copasi_file` as an argument.

In [6]:
zi.save()

##equivalent
zi.save(copasi_file=zi_path)

Model(name=Zi2007_TGFbeta_signaling, time_unit=min, volume_unit=l, quantity_unit=nmol)

# Get Model Information 
## Model attributes
Global model information such as model units or components are available as python attributes. 

In [7]:
attributes = ['time_unit', 'name', 'volume_unit', 'quantity_unit', 'area_unit', 
        'length_unit']
for attr in attributes:
    print ('is {} in zi model attributes? --> {}'.format(attr, attr in dir(zi)))


is time_unit in zi model attributes? --> True
is name in zi model attributes? --> True
is volume_unit in zi model attributes? --> True
is quantity_unit in zi model attributes? --> True
is area_unit in zi model attributes? --> True
is length_unit in zi model attributes? --> True


## Access Model Attributes
Lists of `metabolites`, `global_quantities`, `functions`, `reactions`, `local_parameters` and `compartments` are all available as `model.Model` attributes. For example

In [8]:
## metabolites
print (type(zi.metabolites), len(zi.metabolites))

(<type 'list'>, 16)


In [9]:
## reactions
print (type(zi.reactions), len(zi.reactions))

(<type 'list'>, 26)


Each element of the list returned from an attribute is a pycotools object.

In [10]:
print(type(zi.metabolites[0]))
zi.metabolites[0]

<class 'pycotools.model.Metabolite'>


Metabolite(name="Smad3n", key="Metabolite_3", compartment="Nucleus", concentration="236.45", particle_number="4.98377321973e+13", simulation_type="reactions")

There are objects for `GlobalQuantity`, `Function`, `Reaction`, `LocalParameter` and `Compartment`. classes. These classes are storage classes and relevant information about that class is available as a python attribute.

In [11]:
##get global quantities list
list_of_global_quantities = zi.global_quantities
print ('type list_of_global_quantities --> {}'.format(type(list_of_global_quantities)))

##get first GlobalQuantity in list
global_quantity = list_of_global_quantities[0]
print ('type global_quantity --> {}'.format(type(global_quantity)))

## name of first GlobalQuantity
print ('First global quantity in the list is --> {}'.format(global_quantity.name))

## starting value of first GlobalQuantity
print ('First global quantity in the list has initial value of --> {}'.format(global_quantity.initial_value))

type list_of_global_quantities --> <type 'list'>
type global_quantity --> <class 'pycotools.model.GlobalQuantity'>
First global quantity in the list is --> Kexp_Smad2n
First global quantity in the list has initial value of --> 1.0


# Get, Set, Add and Remove
## How to get model objects
The `Model.get` method is a high level interface into pycotools model components. All types of pycotools component and their attributes are supported. Refer to the documentation for which model components contain which attributes. 

Here are some examples:

### Get the Smad3c metabolite

In [12]:
## get smad3 metabolite by name
print (zi.get('metabolite', 'Smad3c', by='name'))

Metabolite(name="Smad3c", key="Metabolite_1", compartment="Cytoplasm", concentration="492.610000001", particle_number="3.11489514795e+14", simulation_type="reactions")


### Get any global quantity with a fixed simulation_type attribute 

In [13]:
## get qlobal quantities with fixed simulation type
print (zi.get('global_quantity', 'fixed', by='simulation_type'))

[GlobalQuantity(name='Kexp_Smad2n', key='ModelValue_12', initial_value='1.0', simulation_type='fixed'), GlobalQuantity(name='Kdeg_T1R_EE', key='ModelValue_9', initial_value='0.005', simulation_type='fixed'), GlobalQuantity(name='Klid', key='ModelValue_8', initial_value='0.02609', simulation_type='fixed'), GlobalQuantity(name='kr_EE', key='ModelValue_3', initial_value='0.033', simulation_type='fixed'), GlobalQuantity(name='ki_EE', key='ModelValue_2', initial_value='0.33', simulation_type='fixed'), GlobalQuantity(name='v_T2R', key='ModelValue_1', initial_value='0.02869', simulation_type='fixed'), GlobalQuantity(name='v_T1R', key='ModelValue_0', initial_value='0.0103', simulation_type='fixed'), GlobalQuantity(name='k_LRC', key='ModelValue_7', initial_value='2197.0', simulation_type='fixed'), GlobalQuantity(name='Kcd', key='ModelValue_6', initial_value='0.005', simulation_type='fixed'), GlobalQuantity(name='kr_Cave', key='ModelValue_5', initial_value='0.03742', simulation_type='fixed'), Gl

### Get a function by its expression

In [14]:
print (zi.get('function', 'v', by='expression'))

Function(name="Constant flux (irreversible)", key="Function_6", expression="v", roles={})


### Get all local parameters in the R17_LRC_formation reaction

In [15]:
print (zi.get('local_parameter', 'R17_LRC_formation', by='reaction_name'))

LocalParameter(name="k_LRC", reaction_name="R17_LRC_formation, value="2197", simulation_type="fixed")


## How to change existing model attributes
Like `get` the `Model.set` method is a high level interface that takes a model component as first argument. The `set` method first searches the model for a component using the `get` method and then changes an attribute of the returned object. Often the attribute you want to change is the same as the attribute you've used to locate the object of interest (i.e. changing the name of a component). This does not need to be the case however. Here are some examples:

### Change the name of a metabolite

In [16]:
## before the change
print(zi.get('metabolite', 'Smad3c', by='name'))

## Set the name of the metabolite with name 'Smad3c to 'NewSmad3c  
zi.set('metabolite', 'Smad3c', 'NewSmad3c', match_field='name', change_field='name')

## get Smad3c after the change
print( zi.get('metabolite', 'Smad3c', by='name')) ## returns [] since it now doesn't exist

## get NewSmad3c after the change
print(zi.get('metabolite', 'NewSmad3c', by='name')) ##default value for by arg is 'name'

## Change the name back
zi.set('metabolite', 'NewSmad3c', 'Smad3c', match_field='name', change_field='name')
## get Smad3c again after the change back
print(zi.get('metabolite', 'Smad3c', by='name'))

Metabolite(name="Smad3c", key="Metabolite_1", compartment="Cytoplasm", concentration="492.610000001", particle_number="3.11489514795e+14", simulation_type="reactions")
[]
Metabolite(name="NewSmad3c", key="Metabolite_1", compartment="Cytoplasm", concentration="492.610000001", particle_number="3.11489514795e+14", simulation_type="reactions")
Metabolite(name="Smad3c", key="Metabolite_1", compartment="Cytoplasm", concentration="492.610000001", particle_number="3.11489514795e+14", simulation_type="reactions")


### Change initial_value of a global_quantity

In [17]:
## get Kexp_Smad2n global before a change
kexp_smad2n = zi.get('global_quantity', 'Kexp_Smad2n', by='name')
print(kexp_smad2n)

## keep the initial value for later
original_initial_value = kexp_smad2n.initial_value

## change initial_value
print (zi.set('global_quantity', 'Kexp_Smad2n', 35, match_field='name', change_field='initial_value'))
zi.open()
## get Kexp_Smad2n after the change
print (zi.get('global_quantity', 'Kexp_Smad2n', by='name'))

## change back to original value
zi.set('global_quantity', 'Kexp_Smad2n', original_initial_value, match_field='name', change_field='initial_value')

GlobalQuantity(name='Kexp_Smad2n', key='ModelValue_12', initial_value='1.0', simulation_type='fixed')
Model(name=Zi2007_TGFbeta_signaling, time_unit=min, volume_unit=l, quantity_unit=nmol)
GlobalQuantity(name='Kexp_Smad2n', key='ModelValue_12', initial_value='35.0', simulation_type='fixed')


Model(name=Zi2007_TGFbeta_signaling, time_unit=min, volume_unit=l, quantity_unit=nmol)

## How to add a model component
It is possible to add components to a copasi model directly from pycotools. However, the best way to add components is still to use the copasi GUI (easily accessable using the `Model.open` method). 

To add a component to the model directly from python, first create an instance of the class for the component you want to add. The possibilities are `model.Compartment`, `model.Metabolite`, `model.GlobalQuantity`, `model.Reaction`, `model.Function`. 

Note that support has not been built for assigning global_quantities to rate parameters for reactions. This is on the TODO list. 

### Add a metabolite

In [18]:
## create a model.Metabolite object with desired attributes
metab = model.Metabolite(zi, 'metab', concentration=10, compartment=zi.compartments[1])

## add metabolite to model 
zi = zi.add('metabolite', metab)

## Check the metabolite exists
zi.get('metabolite', 'metab', by='name')

Metabolite(name="metab", key="Metabolite_10000", compartment="Nucleus", concentration="10.0", particle_number="2.10774929995e+12", simulation_type="reactions")

Alternatively create a metabolite with default attribute values

In [19]:
zi = zi.add('metabolite', 'this_is_a_new_metabolite')

zi.get('metabolite', 'this_is_a_new_metabolite')

Metabolite(name="this_is_a_new_metabolite", key="Metabolite_10001", compartment="Medium", concentration="1.0", particle_number="6.022140857e+14", simulation_type="reactions")

### Add a global quantity to the model

In [25]:
## Create GlobalQuantity instance
x_global = model.GlobalQuantity(zi, 'X', initial_value=25)

## add to the model
zi.add('global_quantity', x_global)

## Get the global quantity
zi.get('global_quantity', 'X')

[GlobalQuantity(name='X', key='ModelValue_10000', initial_value='25.0', simulation_type='fixed'),
 GlobalQuantity(name='X', key='ModelValue_10001', initial_value='25.0', simulation_type='fixed')]

## How to change existing model attributes
 

### Add a component
The `Model.add` method is a factory method that takes a model component as first argument and then the component second. 

In [20]:
## create instance of Compartment class
fake_compartment = model.Compartment(
    zi, 
    name='fake_compartment',
    initial_value='1'
)
## add compartment to model. Adding a component does not happen inplace
zi = zi.add('compartment', fake_compartment)

print (zi.get('compartment', 'fake_compartment', by='name'))

Compartment(name=fake_compartment, key=Compartment_10000, initial_value=1.0)


The same works for a Metabolite and the other model components

In [21]:
## New metabolite
fake_metabolite = model.Metabolite(
    zi,
    name='FakeMetabolite',
    compartment='fake_compartment',
    concentration=15
)
## add to model
zi = zi.add('metabolite', fake_metabolite)

##save
zi.save()
# print (zi.get('metabolite', 'FakeMetabolite'))


Model(name=Zi2007_TGFbeta_signaling, time_unit=min, volume_unit=l, quantity_unit=nmol)

Check metabolite was added

In [22]:
zi.get('metabolite', 'FakeMetabolite')

Metabolite(name="FakeMetabolite", key="Metabolite_10002", compartment="fake_compartment", concentration="15.0", particle_number="9.0332112855e+15", simulation_type="reactions")

## Remove model components
Similarly to `get` and `set` there is a `remove` function